In [1]:
!pip uninstall -y openai
!pip install --quiet openai==0.28.0


Found existing installation: openai 1.106.1
Uninstalling openai-1.106.1:
  Successfully uninstalled openai-1.106.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00


In [29]:
import openai
import getpass

api_key = getpass.getpass("Enter your GROQ API Key: ")
openai.api_key = api_key
openai.api_base = "https://api.groq.com/openai/v1"


Enter your GROQ API Key: ··········


In [30]:
class ConversationManager:
    def __init__(self, max_turns=5, max_chars=1000, summarize_every=3, model="llama-3-8b-8192"):
        self.history = []
        self.max_turns = max_turns
        self.max_chars = max_chars
        self.summarize_every = summarize_every
        self.run_count = 0
        self.model = model

    def add_message(self, role, content):
        self.history.append({"role": role, "content": content})
        self.run_count += 1
        print(f"Added message ({role}): {content}")

        # Summarize after every k-th run
        if self.run_count % self.summarize_every == 0:
            self.summarize_history()
        self.truncate_history()

    def truncate_history(self):
        # By number of turns
        if len(self.history) > self.max_turns:
            self.history = self.history[-self.max_turns:]
        # By total character length
        total_chars = sum(len(msg["content"]) for msg in self.history)
        while total_chars > self.max_chars and len(self.history) > 1:
            self.history.pop(0)
            total_chars = sum(len(msg["content"]) for msg in self.history)

    def summarize_history(self):
        all_text = "\n".join(f'{msg["role"]}: {msg["content"]}' for msg in self.history)
        print("\nSummarizing history...")
        summary = self.get_summary(all_text)
        self.history = [{"role": "system", "content": f"Summary so far: {summary}"}]
        print(f"Summary stored: {summary}")

    def get_summary(self, text):
        response = openai.ChatCompletion.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "Summarize the following conversation in 2-3 sentences."},
                {"role": "user", "content": text}
            ],
            max_tokens=150,
            temperature=0.3
        )
        return response.choices[0].message.content.strip()

    def show_history(self):
        print("\nCurrent Conversation History:")
        for i, msg in enumerate(self.history):
            print(f"{i+1}. {msg['role']}: {msg['content']}")


In [31]:
# Demo messages for testing
samples = [
    ("user", "Hi, I need help booking a flight to Delhi."),
    ("assistant", "Sure! What date are you planning to travel?"),
    ("user", "I'm looking at the 25th of September."),
    ("assistant", "Do you prefer morning or evening flights?"),
    ("user", "Morning would be better."),
    ("assistant", "Great! I'll check available morning flights for 25th September to Delhi."),
    ("user", "Please check if there are any discounts available."),
]

manager = ConversationManager(
    max_turns=5,
    max_chars=1000,
    summarize_every=3,
    model="llama-3.1-8b-instant"
)

for role, content in samples:
    manager.add_message(role, content)
    manager.show_history()
    print("-" * 40)


Added message (user): Hi, I need help booking a flight to Delhi.

Current Conversation History:
1. user: Hi, I need help booking a flight to Delhi.
----------------------------------------
Added message (assistant): Sure! What date are you planning to travel?

Current Conversation History:
1. user: Hi, I need help booking a flight to Delhi.
2. assistant: Sure! What date are you planning to travel?
----------------------------------------
Added message (user): I'm looking at the 25th of September.

Summarizing history...
Summary stored: There is no information provided about the conversation after the user's date of travel.

Current Conversation History:
1. system: Summary so far: There is no information provided about the conversation after the user's date of travel.
----------------------------------------
Added message (assistant): Do you prefer morning or evening flights?

Current Conversation History:
1. system: Summary so far: There is no information provided about the conversatio

# Task 2

In [32]:
extraction_schema = {
    "name": "extract_user_details",
    "description": "Extract user details like name, email, phone, location, age from a chat.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "Full name of the user"
            },
            "email": {
                "type": "string",
                "description": "User's email address",
                "pattern": "^[\\w\\.-]+@[\\w\\.-]+\\.\\w+$"
            },
            "phone": {
                "type": "string",
                "description": "User's phone number",
                "pattern": "^\\+?[0-9\\-\\s]{7,15}$"
            },
            "location": {
                "type": "string",
                "description": "User's location"
            },
            "age": {
                "type": "integer",
                "description": "User's age"
            }
        },
        "required": ["name", "email", "phone", "location", "age"]
    }
}


In [33]:
import openai
import getpass

api_key = getpass.getpass("Enter your GROQ API Key: ")

openai.api_key = api_key
openai.api_base = "https://api.groq.com/openai/v1"


Enter your GROQ API Key: ··········


In [34]:
import re

def mask_sensitive(text):
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b', '[EMAIL]', text)
    text = re.sub(r'\b\d{10}\b|\+\d{1,3}-\d{3,4}-\d{4}|\d{3,4}-\d{4}', '[PHONE]', text)
    text = re.sub(r'((my name is|name is|this is|I am)\s+)([A-Z][a-z]+(\s+[A-Z][a-z]+)?)', r'\1[NAME]', text)
    text = re.sub(r'^(Hi, my name is|Hello, this is)\s+[A-Z][a-z]+(\s+[A-Z][a-z]+)?', r'\1 [NAME]', text)
    text = re.sub(r'(in|from|location:)\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)', r'\1 [LOCATION]', text)
    return text

sample_chats = [
    "Hi, my name is Alice Johnson. You can email me at alice.johnson@example.com or call me at +1-555-1234. I live in New York and I am 29 years old.",
    "I'm Bob! My email: bob.smith@company.org, phone: 9876543210, location: London, age: 35.",
    "Hello, this is Carol. Reach me at carol@domain.com, contact: 555-9876, from Mumbai, age 42."
]

masked_chats = [mask_sensitive(chat) for chat in sample_chats]

sample_chats_structured = [
    {
        "name": "Alice Johnson",
        "email": "alice.johnson@example.com",
        "phone": "+1-555-1234",
        "location": "New York",
        "age": 29
    },
    {
        "name": "Bob",
        "email": "bob.smith@company.org",
        "phone": "9876543210",
        "location": "London",
        "age": 35
    },
    {
        "name": "Carol",
        "email": "carol@domain.com",
        "phone": "555-9876",
        "location": "Mumbai",
        "age": 42
    }
]

print(masked_chats)
print(sample_chats_structured)


['Hi, my name is [NAME]. You can email me at [EMAIL] or call me at [PHONE]. I live in [LOCATION] and I am 29 years old.', "I'm Bob! My email: [EMAIL], phone: [PHONE], location: [LOCATION], age: 35.", 'Hello, this is [NAME]. Reach me at [EMAIL], contact: [PHONE], from [LOCATION], age 42.']
[{'name': 'Alice Johnson', 'email': 'alice.johnson@example.com', 'phone': '+1-555-1234', 'location': 'New York', 'age': 29}, {'name': 'Bob', 'email': 'bob.smith@company.org', 'phone': '9876543210', 'location': 'London', 'age': 35}, {'name': 'Carol', 'email': 'carol@domain.com', 'phone': '555-9876', 'location': 'Mumbai', 'age': 42}]


In [35]:
import json

def extract_details(chat_text):
    try:
        response = openai.ChatCompletion.create(
            model="llama-3.1-8b-instant",
            messages=[
                {"role": "system", "content": "Extract user details from the following chat."},
                {"role": "user", "content": chat_text}
            ],
            functions=[extraction_schema],
            function_call={"name": "extract_user_details"}
        )
        args = response.choices[0].message.function_call.arguments
        if isinstance(args, str):
            return json.loads(args)
        return args
    except Exception as e:
        print(f"Error extracting details: {e}")
        return {}

for i, chat in enumerate(sample_chats, 1):
    print(f"\n--- Sample Chat {i} ---")
    print(chat)
    details = extract_details(chat)
    print("\nExtracted Details (JSON):")
    print(json.dumps(details, indent=2))
    # Validate required keys
    required_keys = ["name", "email", "phone", "location", "age"]
    valid = all(k in details for k in required_keys)
    print(f"Schema validation: {'Passed' if valid else ' Failed'}")



--- Sample Chat 1 ---
Hi, my name is Alice Johnson. You can email me at alice.johnson@example.com or call me at +1-555-1234. I live in New York and I am 29 years old.

Extracted Details (JSON):
{
  "age": 29,
  "email": "alice.johnson@example.com",
  "location": "New York",
  "name": "Alice Johnson",
  "phone": "+1-555-1234"
}
Schema validation: Passed

--- Sample Chat 2 ---
I'm Bob! My email: bob.smith@company.org, phone: 9876543210, location: London, age: 35.

Extracted Details (JSON):
{
  "age": 35,
  "email": "bob.smith@company.org",
  "location": "London",
  "name": "Bob",
  "phone": "9876543210"
}
Schema validation: Passed

--- Sample Chat 3 ---
Hello, this is Carol. Reach me at carol@domain.com, contact: 555-9876, from Mumbai, age 42.

Extracted Details (JSON):
{
  "age": 42,
  "email": "carol@domain.com",
  "location": "Mumbai",
  "name": "Carol",
  "phone": "555-9876"
}
Schema validation: Passed
